# API for Game Data

In [3]:
import requests

def get_json(url):
    headers = {"User-Agent": "xcs5hg@virginia.edu"}
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    return r.json()

season_data = get_json("https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024")

season_types = season_data["types"]["items"]

reg_season = next(t for t in season_types if t["name"] == "Regular Season")

weeks_url = reg_season["weeks"]["$ref"]
print("Weeks URL:", weeks_url)

weeks_data = get_json(weeks_url)
print(weeks_data.keys())

Weeks URL: http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/types/2/weeks?lang=en&region=us
dict_keys(['count', 'pageIndex', 'pageSize', 'pageCount', 'items'])


In [ ]:
weeks_items = weeks_data["items"]

all_game_urls = []

for w in weeks_items:
    week_url = w["$ref"]
    week = get_json(week_url)
    
    events_url = week["events"]["$ref"]
    events = get_json(events_url)["items"]  
    
    for e in events:
        all_game_urls.append(e["$ref"])

print("Number of games found:", len(all_game_urls))

Number of games found: 272


In [10]:
all_game_urls = all_game_urls[:3]

In [ ]:
import pandas as pd

games_data = []

for game_url in all_game_urls:
    game = get_json(game_url)
    
    comp = game["competitions"][0]
    date = comp.get("date")
    
    competitors = comp["competitors"]
    home_comp = next(c for c in competitors if c.get("homeAway") == "home")
    away_comp = next(c for c in competitors if c.get("homeAway") == "away")
    
    home_team_url = home_comp["team"]["$ref"]
    away_team_url = away_comp["team"]["$ref"]
    
    home_team_data = get_json(home_team_url)
    away_team_data = get_json(away_team_url)
    
    games_data.append({
        "game_id": game.get("id"),
        "date": date,
        "home_team": home_team_data.get("displayName") or home_team_data.get("shortDisplayName"),
        "home_abbr": home_team_data.get("abbreviation"),
        "away_team": away_team_data.get("displayName") or away_team_data.get("shortDisplayName"),
        "away_abbr": away_team_data.get("abbreviation"),
        "status": comp.get("status", {}).get("type", {}).get("name")
    })

df_schedule = pd.DataFrame(games_data)
print(df_schedule.head())

     game_id               date            home_team home_abbr  \
0  401671789  2024-09-06T00:40Z   Kansas City Chiefs        KC   
1  401671805  2024-09-07T00:15Z  Philadelphia Eagles       PHI   
2  401671744  2024-09-08T17:00Z      Atlanta Falcons       ATL   

             away_team away_abbr status  
0     Baltimore Ravens       BAL   None  
1    Green Bay Packers        GB   None  
2  Pittsburgh Steelers       PIT   None  


# Odds API

In [12]:
import requests

header = {"User-Agent": "xcs5hg@virginia.edu"}

def get_json(url):
    r = requests.get(url, headers=header)
    r.raise_for_status()
    return r.json()

In [13]:
def get_win_probabilities(game_id):

    event_url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{game_id}"
    event = get_json(event_url)

    comp = event["competitions"][0]
    comp_id = comp["id"]

    probs_url = (
        f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/"
        f"events/{game_id}/competitions/{comp_id}/probabilities?limit=200"
    )

    probs = get_json(probs_url)
    return probs

In [14]:
def get_odds(game_id):
    """
    Fetch odds (e.g., spreads, moneylines, totals) for a given NFL game.
    Returns the raw JSON from the odds endpoint.
    """
    # 1) Get the event to find the competition id
    event_url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{game_id}"
    event = get_json(event_url)

    comp = event["competitions"][0]
    comp_id = comp["id"]

    # 2) Build odds endpoint
    odds_url = (
        f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/"
        f"events/{game_id}/competitions/{comp_id}/odds"
    )

    odds = get_json(odds_url)
    return odds

In [15]:
import re

def extract_game_id(event_url: str) -> str:
    # grabs the numeric id from the URL
    return re.search(r"/events/(\d+)", event_url).group(1)

game_ids = [extract_game_id(u) for u in all_game_urls]

# Example: get odds + probabilities for the first game
test_id = game_ids[0]
test_probs = get_win_probabilities(test_id)
test_odds = get_odds(test_id)

print("Game ID:", test_id)
print("Probabilities count:", test_probs.get("count"))
print("Odds count:", test_odds.get("count"))

Game ID: 401671789
Probabilities count: 188
Odds count: 3


In [16]:
test_probs

{'count': 188,
 'pageIndex': 1,
 'pageSize': 200,
 'pageCount': 1,
 'items': [{'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/401671789/competitions/401671789/probabilities/4016717891?lang=en&region=us',
   'competition': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/401671789/competitions/401671789?lang=en&region=us'},
   'play': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/401671789/competitions/401671789/plays/4016717891?lang=en&region=us'},
   'homeTeam': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/teams/12?lang=en&region=us'},
   'awayTeam': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/teams/33?lang=en&region=us'},
   'tiePercentage': 0.0,
   'homeWinPercentage': 0.5833,
   'awayWinPercentage': 0.41669999999999996,
   'lastModified': '2024-09-06T05:53Z',
   'sequenceNumber': '100',
   'source': {'id'